In [233]:
import numpy as np
import math

t = 0  # number of this round
T = 8000  # numbers of rounds
l = 30  # ||a|| <= L
d = 3 #dimension of arm
m = 1 #||theta|| < m
theta_stimato = [0.0, 0.0, 0.0] #np.array([])  theta
A = [] #collector of arm
num_arms = 0 #number of arms
num_theta = 0 #num theta
Th = [] #collector of theta
W = [] #distribution of probability to play arms
N = [] #number of times an arm is played
R = [] #collection of reward
H = [] #collection of hedge losses
L = [] #collection of losses
Mixability_Gap = [] #collection of all mixability gap at each t
AI = [] #number of the arm played at time I
nu = 0.01 #regularization paramether
eta = 0.1 #learning paramether
theta_vero = [0]*6
curr_answ = 0

Y = []
num_arms_y = 0

M = [] #world
num_M = 0

class world():
    matrix = np.array([])
    id

    def __init__(self, param):
        self.matrix = param
        global M, num_M
        self.id = num_M
        M.append(self)
        num_M +=1
        
class arm_y():
    arm_vector = np.array([])
    id = 0

    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms_y, arm_Y
        Y.append(self)
        self.id = num_arms_y
        num_arms_y += 1
        
def create_theta():
    for i in range (0, num_arms_y):
        new_theta = theta(np.matmul(M[0].matrix, Y[i].arm_vector))

def normalize(v):
    sum = np.sum(v)
    if sum > 0:
        return v / sum
    else:
        return np.array([0.33]*len(v))
    
class theta():
    vector = np.array([])
    best_arm = -1
    id

    def __init__(self, param):
        self.vector = param
        global Th, num_theta
        self.id = num_theta
        Th.append(self)
        num_theta +=1
        #i = 0
        #best = -5
        #while i < num_arms:
            #tmp = np.dot( A[i].arm_vector, np.transpose(self.vector))
            #if best < tmp:
                #best = tmp
                #self.best_arm = i

class arm():
    arm_vector = np.array([])
    id = 0
    best_theta = -1
    regret_y = []

    def __init__(self, vector):
        self.regret_y = [0] *num_arms_y
        self.arm_vector = vector
        global num_arms, A
        A.append(self)
        W.append(0)
        N.append(0)
        L.append(0)
        self.id = num_arms
        num_arms +=1
        max = -5
        for i in range (0, num_theta):
            tmp = np.matmul(self.arm_vector, Th[i].vector)
            if tmp > max:
                self.best_theta = i
                max = tmp
                            
    def new_theta_vero(self):
        min = 1000000000000000
        curr = -1
        for i in range (0, num_arms_y):
            if self.regret_y[i] < min:
                curr = i
                min = self.regret_y[i]
        global theta_vero
        theta_vero[self.id] = curr
                    
    def pull_arm(self):
        r = np.dot(self.arm_vector, np.transpose(Th[theta_vero[self.id]].vector)) + np.random.normal(0, 1)
        R.append(r)
        print("self.id", self.id)
        print("theta_vero", theta_vero)
        print("regret_y", self.regret_y)
        self.regret_y[theta_vero[self.id]] += (4*(l**2)*(m**2) - r)/1000
        self.new_theta_vero()
        N[self.id] += 1
        AI.append(self.id)
        return r
        
    def norm(self):
        return np.linalg.norm(self.arm_vector)

def design_matrix(w):
    V = []
    i = 0
    while i < num_arms:
        b = [A[i].arm_vector]
        a = np.swapaxes(b, 0, 1)
        ris = np.matmul(a, b) * w[i]
        if (i == 0):
            V = ris
        else:
            V += ris
        i+=1
    return V

def norm_2_mat(x, V):
    b = [x]
    a = np.swapaxes(b, 0, 1)
    ris = np.sqrt(np.matmul(np.matmul(b,V),a))
    return ris[0][0]

def arm_to_play():
    diff = np.array(N) - np.array(W)
    i = 0
    arm = 0
    min = +5000
    while i < num_arms:
        if diff[i] < min:
            min = diff[i]
            arm = i
        i += 1
    return arm

def stima_theta():
    V = design_matrix(N)
    I = np.identity(d)
    p = np.linalg.inv(V + nu*I)
    sum = 0
    for i in range (0, t):
        sum += R[i] * np.array(A[AI[i]].arm_vector)
    res = np.matmul(p, sum)
    global theta_stimato
    theta_stimato = res
    return res

def best_answer():
    max = -100
    best = -1
    for i in range (0, num_arms):
        curr = np.matmul(A[i].arm_vector, theta_stimato)
        if curr > max:
            max = curr
            best = i
    global curr_answ
    curr_answ = best
    return best

def best_arm():
    max = -100
    best = -1
    for i in range (0, num_arms):
        curr = np.matmul(A[i].arm_vector, Th[theta_vero[i]].vector)
        if curr > max:
            max = curr
            best = i
    global curr_answ
    curr_answ = best
    return best


def beta(time, delta):
    n = np.abs(np.random.normal(0, 1))
    res = math.sqrt(math.log10(1/delta) + d/2*math.log10(1 + (time*(l**2))/(n*d))) + math.sqrt(n/2)*m
    return res

def best_response():
    min = 1000000
    theta_adv = -11
    for i in range (0, num_theta):
        if not i == A[curr_answ].best_theta:
            curr = (norm_2_mat(np.array(theta_stimato) - np.array(Th[i].vector), design_matrix(W)))**2
            if curr < min:
                min = curr
                theta_adv = i
    return theta_adv

def calcola_U (id_arm):
    xi = [0]*d
    q2 = 4*(l**2)*(m**2)
    q1 = 0
    p1 = np.matmul((np.array(theta_stimato) - np.array(Th[best_response()].vector)), np.array(A[id_arm].arm_vector))
    p2 = math.sqrt(2*beta(T, 1/T**3))*norm_2_mat(np.array(A[id_arm].arm_vector), design_matrix(normalize(N)) + np.random.normal(0, 1)*np.identity(d))
    r1 = (p1 + p2)**2
    r2 = (p1 - p2)**2
    if r1 > r2:
        q1 = r1
    else:
        q1 = r2
    if q1 > q2:
        return q2
    else:
        return q1

def calcola_loss():
    loss = 0
    arm_play = arm_to_play()
    best_res = best_response ()
    for i in range (0, num_arms):
        loss += W[i]*calcola_U(i)/2
    L[arm_play] += (4*(l**2)*(m**2) - loss)/1000
    return (4*(l**2)*(m**2) - loss)/1000

In [234]:
len([0,1])

2

In [235]:

army2 = arm_y([0.4, 0.6, 0.2])
army1 = arm_y([0.1, 0.4, 0.5])
army3 = arm_y([0.01, 0.1, 0.8])

In [236]:
wor = world( [[0.8730240451816763, 0.21159403045606273, 0.28918565354173686], [0.5189626386885673, 0.4849994006831272, 0.9629953198577507], [0.17670422753928094, 0.2532554211216236, 0.9072025363404456]] )


In [237]:
create_theta()

In [238]:
a = arm([1,2,3])
b = arm([2,5,1])
c = arm([1, 3, 1])

In [239]:
num_theta

3

In [240]:
Th[2].vector

array([0.26123817, 0.82408582, 0.75285461])

In [241]:
w = [0.34, 0.33, 0.33]
V = design_matrix(w)

In [242]:
np.linalg.norm(c.arm_vector)

3.3166247903554

In [243]:
for i in range (0, num_arms):
    print(A[i].best_theta)

2
2
2


In [244]:
for i in range(0, num_arms):
    for j in range (0, num_theta):
        print(np.matmul(A[i].arm_vector, Th[j].vector))

3.1285970396386893
3.4890417897318278
4.167973651393345
4.928000584350354
4.842607966670638
5.3957600578285305
3.011629897581991
3.07128775505721
3.4863502468149212


In [245]:
arm_to_play()

0

In [246]:
for i in range (0, num_arms):
    print(np.matmul(A[i].arm_vector, Th[theta_vero[i]].vector))

3.1285970396386893
4.928000584350354
3.011629897581991


In [247]:
def h():
    return np.dot(W, L)

def mix_loss():
    sum = 0
    for i in range (0, num_arms):
        sum += W[i] * math.exp(-eta * L[i])
    return -1/eta * np.log(sum)

def mixability_gap():
    m_g = h() - mix_loss()
    Mixability_Gap.append(m_g)

def new_eta():
    if t>0:
        sum = 0
        for i in range (0, t):
            sum += Mixability_Gap[i]
        eta = math.log(d)/sum
        
def generate_w():
    sum=0
    for i in range (0, num_arms):
        W[i] =math.exp(-eta * L[i])/num_arms
        sum += math.exp(-eta * L[i])/num_arms
    for i in range (0, num_arms):
        W[i] = W[i]/sum
    return W

In [248]:
a.pull_arm()

self.id 0
theta_vero [0, 0, 0, 0, 0, 0]
regret_y [0, 0, 0]


3.289233188214264

In [249]:
t += 1
stima_theta()
best_answer()
curr = arm_to_play()
mixability_gap()
generate_w()
err = calcola_loss()
print(err)
A[curr].pull_arm()

2.7283945818155297
self.id 1
theta_vero [1, 0, 0, 0, 0, 0]
regret_y [0, 0, 0]


D:\Programmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


5.195150238191807

In [250]:
t += 1
stima_theta()
best_answer()
curr = arm_to_play()
mixability_gap()
new_eta()
generate_w()
err = calcola_loss()
print(err)
A[curr].pull_arm()

2.291442019291936
self.id 2
theta_vero [1, 1, 0, 0, 0, 0]
regret_y [0, 0, 0]


1.4559535742157048

In [251]:
N

[1, 1, 1]

In [252]:
normalize(N)

array([0.33333333, 0.33333333, 0.33333333])

In [253]:
for i in range (800):
    t += 1
    stima_theta()
    best_answer()
    curr = arm_to_play()
    mixability_gap()
    new_eta()
    generate_w()
    err = calcola_loss()
    print(err)
    A[curr].pull_arm()

2.418925009584693
self.id 0
theta_vero [1, 1, 1, 0, 0, 0]
regret_y [3.5967107668117855, 0, 0]
2.39938340555448
self.id 2
theta_vero [2, 1, 1, 0, 0, 0]
regret_y [3.5985440464257845, 0, 0]
2.40786727002571
self.id 1
theta_vero [2, 1, 2, 0, 0, 0]
regret_y [3.5948048497618084, 0, 0]
2.4153063920635933
self.id 0
theta_vero [2, 2, 2, 0, 0, 0]
regret_y [3.5967107668117855, 3.5960377277089264, 0]
2.377502453174112
self.id 2
theta_vero [2, 2, 2, 0, 0, 0]
regret_y [3.5985440464257845, 3.597136055727268, 0]
2.40391749398519
self.id 1
theta_vero [2, 2, 2, 0, 0, 0]
regret_y [3.5948048497618084, 3.595458499802831, 0]
2.36389775658672
self.id 0
theta_vero [2, 2, 2, 0, 0, 0]
regret_y [3.5967107668117855, 3.5960377277089264, 3.5955158448541287]
2.421601112211525
self.id 2
theta_vero [1, 2, 2, 0, 0, 0]
regret_y [3.5985440464257845, 3.597136055727268, 3.594765903577567]
2.4699475924760272
self.id 1
theta_vero [1, 2, 1, 0, 0, 0]
regret_y [3.5948048497618084, 3.595458499802831, 3.594140103847401]
2.3843842

2.3853291940792998
self.id 2
theta_vero [0, 1, 2, 0, 0, 0]
regret_y [32.37638145344715, 32.37524162818224, 32.364455585102945]
2.266094393267942
self.id 1
theta_vero [0, 1, 1, 0, 0, 0]
regret_y [32.35597345979284, 32.35449857024218, 35.94801023108588]
2.3602725141515437
self.id 0
theta_vero [0, 0, 1, 0, 0, 0]
regret_y [32.36974176628352, 32.37141023001039, 35.95752371559432]
2.3645170271686196
self.id 2
theta_vero [1, 0, 1, 0, 0, 0]
regret_y [32.37638145344715, 32.37524162818224, 35.96061090053038]
2.2771728177990935
self.id 1
theta_vero [1, 0, 0, 0, 0, 0]
regret_y [32.35597345979284, 35.94910333646511, 35.94801023108588]
2.4158097367613554
self.id 0
theta_vero [1, 2, 0, 0, 0, 0]
regret_y [35.96598256132408, 32.37141023001039, 35.95752371559432]
2.3220451556488975
self.id 2
theta_vero [2, 2, 0, 0, 0, 0]
regret_y [32.37638145344715, 35.97011689024907, 35.96061090053038]
2.293038813913833
self.id 1
theta_vero [2, 2, 2, 0, 0, 0]
regret_y [35.95113821009339, 35.94910333646511, 35.948010231

2.288591915856061
self.id 1
theta_vero [2, 2, 2, 0, 0, 0]
regret_y [79.09119785447776, 79.0940330770277, 79.08643929657562]
2.308250329288828
self.id 0
theta_vero [2, 0, 2, 0, 0, 0]
regret_y [79.1235006061897, 79.11940065469393, 79.10361804369086]
2.300632206734335
self.id 2
theta_vero [1, 0, 2, 0, 0, 0]
regret_y [79.14309280838383, 79.13375227918156, 79.11808008649862]
2.2851392024332804
self.id 1
theta_vero [1, 0, 1, 0, 0, 0]
regret_y [79.09119785447776, 79.0940330770277, 82.68007771944889]
2.2740860771366864
self.id 0
theta_vero [1, 1, 1, 0, 0, 0]
regret_y [79.1235006061897, 79.11940065469393, 82.69821661233351]
2.260403922799125
self.id 2
theta_vero [0, 1, 1, 0, 0, 0]
regret_y [79.14309280838383, 79.13375227918156, 82.71344479134976]
2.330647908121768
self.id 1
theta_vero [0, 1, 0, 0, 0, 0]
regret_y [82.68564926954234, 79.0940330770277, 82.68007771944889]
2.370262625260925
self.id 0
theta_vero [0, 2, 0, 0, 0, 0]
regret_y [79.1235006061897, 82.71518738819127, 82.69821661233351]
2.33

2.357950775697742
self.id 0
theta_vero [0, 2, 0, 0, 0, 0]
regret_y [111.49940994888236, 115.08340990196137, 115.06051241224479]
2.3380032703625724
self.id 2
theta_vero [2, 2, 0, 0, 0, 0]
regret_y [111.51776705402689, 115.10726396460963, 115.07801080606016]
2.29029351893555
self.id 1
theta_vero [2, 2, 2, 0, 0, 0]
regret_y [115.04329173136757, 115.04324610962095, 115.03137516891084]
2.315532595141385
self.id 0
theta_vero [2, 1, 2, 0, 0, 0]
regret_y [115.09628918249513, 115.08340990196137, 115.06051241224479]
2.29734604962049
self.id 2
theta_vero [1, 1, 2, 0, 0, 0]
regret_y [115.11486743043504, 115.10726396460963, 115.07801080606016]
2.2989498095652188
self.id 1
theta_vero [1, 1, 1, 0, 0, 0]
regret_y [115.04329173136757, 115.04324610962095, 118.62754741179153]
2.341348900353083
self.id 0
theta_vero [1, 0, 1, 0, 0, 0]
regret_y [115.09628918249513, 115.08340990196137, 118.65572647511537]
2.311715148078549
self.id 2
theta_vero [0, 0, 1, 0, 0, 0]
regret_y [115.11486743043504, 115.107263964609

2.323179596537492
self.id 2
theta_vero [0, 0, 1, 0, 0, 0]
regret_y [140.29376165627147, 140.2884421955009, 143.84973091672222]
2.268133140273844
self.id 1
theta_vero [0, 0, 0, 0, 0, 0]
regret_y [140.20951947386595, 143.80482333370102, 143.7891950728275]
2.296248153652725
self.id 0
theta_vero [0, 2, 0, 0, 0, 0]
regret_y [140.27421623908347, 143.85670887505992, 143.82981961780018]
2.283116620891319
self.id 2
theta_vero [2, 2, 0, 0, 0, 0]
regret_y [140.29376165627147, 143.88648691044028, 143.84973091672222]
2.279323265477782
self.id 1
theta_vero [2, 2, 2, 0, 0, 0]
regret_y [143.80486161500687, 143.80482333370102, 143.7891950728275]
2.319830079822208
self.id 0
theta_vero [2, 1, 2, 0, 0, 0]
regret_y [143.87107686931824, 143.85670887505992, 143.82981961780018]
2.2621408295394274
self.id 2
theta_vero [1, 1, 2, 0, 0, 0]
regret_y [143.88870575871397, 143.88648691044028, 143.84973091672222]
2.220992839807598
self.id 1
theta_vero [1, 1, 1, 0, 0, 0]
regret_y [143.80486161500687, 143.80482333370102

...

2.2382114570556313
self.id 1
theta_vero [0, 1, 0, 0, 0, 0]
regret_y [197.73740141904116, 194.1412966727754, 197.71093396665373]
2.2449427212626043
self.id 0
theta_vero [0, 2, 0, 0, 0, 0]
regret_y [194.22866673018092, 197.79746513643585, 197.7727487560643]
2.2606747449120577
self.id 2
theta_vero [2, 2, 0, 0, 0, 0]
regret_y [194.24281255622526, 197.83338269831435, 197.79812248919018]
2.223105313027115
self.id 1
theta_vero [2, 2, 2, 0, 0, 0]
regret_y [197.73740141904116, 197.73598928352183, 197.71093396665373]
2.3254756756503236
self.id 0
theta_vero [2, 1, 2, 0, 0, 0]
regret_y [197.8261835125691, 197.79746513643585, 197.7727487560643]
2.2972609154924384
self.id 2
theta_vero [1, 1, 2, 0, 0, 0]
regret_y [197.83800487900027, 197.83338269831435, 197.79812248919018]
2.2614534790991354
self.id 1
theta_vero [1, 1, 1, 0, 0, 0]
regret_y [197.73740141904116, 197.73598928352183, 201.3052111020833]
2.2620555403584968
self.id 0
theta_vero [1, 0, 1, 0, 0, 0]
regret_y [197.8261835125691, 197.79746513643

2.2411280666591975
self.id 2
theta_vero [1, 0, 2, 0, 0, 0]
regret_y [230.2096016889126, 230.1983228099924, 230.16395606457348]
2.167554708945231
self.id 1
theta_vero [1, 0, 1, 0, 0, 0]
regret_y [230.09206552205308, 230.09525768139636, 233.6603101452738]
2.287146412102482
self.id 0
theta_vero [1, 1, 1, 0, 0, 0]
regret_y [230.20000655244613, 230.16570249121423, 233.73158120561078]
2.258058185491902
self.id 2
theta_vero [0, 1, 1, 0, 0, 0]
regret_y [230.2096016889126, 230.1983228099924, 233.76158312912574]
2.167626163300516
self.id 1
theta_vero [0, 1, 0, 0, 0, 0]
regret_y [233.68740272391682, 230.09525768139636, 233.6603101452738]
2.225742965910396
self.id 0
theta_vero [0, 2, 0, 0, 0, 0]
regret_y [230.20000655244613, 233.76296920034974, 233.73158120561078]
2.1812414100963147
self.id 2
theta_vero [2, 2, 0, 0, 0, 0]
regret_y [230.2096016889126, 233.7948277538963, 233.76158312912574]
2.2227740760762478
self.id 1
theta_vero [2, 2, 2, 0, 0, 0]
regret_y [233.68740272391682, 233.69177185684606, 2

2.205174787880108
self.id 2
theta_vero [1, 0, 2, 0, 0, 0]
regret_y [262.5811814653304, 262.5731736180972, 262.5314300789383]
2.155862176958992
self.id 1
theta_vero [1, 0, 1, 0, 0, 0]
regret_y [262.445589024344, 262.4485539021631, 266.01112539645874]
2.2512357554025
self.id 0
theta_vero [1, 1, 1, 0, 0, 0]
regret_y [262.5720880536073, 262.53480875346344, 266.0959103700343]
2.223275256419338
self.id 2
theta_vero [0, 1, 1, 0, 0, 0]
regret_y [262.5811814653304, 262.5731736180972, 266.12836166455276]
2.1960095312795898
self.id 1
theta_vero [0, 1, 0, 0, 0, 0]
regret_y [266.04048420478597, 262.4485539021631, 266.01112539645874]
2.217296788812191
self.id 0
theta_vero [0, 2, 0, 0, 0, 0]
regret_y [262.5720880536073, 266.1300900835496, 266.0959103700343]
2.227590417606881
self.id 2
theta_vero [2, 2, 0, 0, 0, 0]
regret_y [262.5811814653304, 266.1716724177377, 266.12836166455276]
2.1996646304527436
self.id 1
theta_vero [2, 2, 2, 0, 0, 0]
regret_y [266.04048420478597, 266.0441161668699, 266.011125396

2.232931672700188
self.id 0
theta_vero [2, 0, 2, 0, 0, 0]
regret_y [291.347710735724, 291.30690313049433, 291.2682657918076]
2.1873183163878474
self.id 2
theta_vero [1, 0, 2, 0, 0, 0]
regret_y [291.35711078182806, 291.34821564767117, 291.3007164107316]
2.1721180253602745
self.id 1
theta_vero [1, 0, 1, 0, 0, 0]
regret_y [291.20402870901046, 291.2054785775022, 294.76938320919646]
2.1771974820170352
self.id 0
theta_vero [1, 1, 1, 0, 0, 0]
regret_y [291.347710735724, 291.30690313049433, 294.8658941149349]
2.176120955653005
self.id 2
theta_vero [0, 1, 1, 0, 0, 0]
regret_y [291.35711078182806, 291.34821564767117, 294.8981236384083]
2.1786993144002356
self.id 1
theta_vero [0, 1, 0, 0, 0, 0]
regret_y [294.79790489256794, 291.2054785775022, 294.76938320919646]
2.1892077643816035
self.id 0
theta_vero [0, 2, 0, 0, 0, 0]
regret_y [291.347710735724, 294.9035369957734, 294.8658941149349]
2.175627487177585
self.id 2
theta_vero [2, 2, 0, 0, 0, 0]
regret_y [291.35711078182806, 294.94440161387416, 294.8

2.1732819471293077
self.id 1
theta_vero [0, 1, 0, 0, 0, 0]
regret_y [319.9663997293132, 316.3717112823524, 319.9350250446155]
2.2200608842558087
self.id 0
theta_vero [0, 2, 0, 0, 0, 0]
regret_y [316.52540615495275, 320.0785045522552, 320.0378873200418]
2.1652246623607754
self.id 2
theta_vero [2, 2, 0, 0, 0, 0]
regret_y [316.5322451483217, 320.1261249520364, 320.07258121530896]
2.140240467894121
self.id 1
theta_vero [2, 2, 2, 0, 0, 0]
regret_y [319.9663997293132, 319.9666488474276, 319.9350250446155]
2.2457328273454693
self.id 2
theta_vero [2, 0, 2, 0, 0, 0]
regret_y [320.128414210862, 320.1261249520364, 320.07258121530896]


In [254]:
print(theta_stimato)
print(Th[theta_vero[best_answer()]].vector)
print(best_answer())
print(best_arm())
print(W)

[-0.11488373  0.91984428  0.62447909]
[0.53400317 0.69118376 0.40407545]
1
1
[0.23042362179056747, 0.5386575940160147, 0.23091878419341794]


In [255]:
print(theta_vero)

[2, 0, 1, 0, 0, 0]


In [256]:
b.pull_arm()
c.pull_arm()
b.pull_arm()

self.id 1
theta_vero [2, 0, 1, 0, 0, 0]
regret_y [319.9663997293132, 319.9666488474276, 323.52858826267067]
self.id 2
theta_vero [2, 1, 1, 0, 0, 0]
regret_y [320.128414210862, 320.1261249520364, 323.6681645059366]
self.id 1
theta_vero [2, 1, 0, 0, 0, 0]
regret_y [323.5631562838478, 319.9666488474276, 323.52858826267067]


3.6222348416383197

In [257]:
R

[3.289233188214264,
 5.195150238191807,
 1.4559535742157048,
 3.9622722910734893,
 2.8639442727322697,
 4.541500197169063,
 4.4841551458714095,
 5.234096422432514,
 5.859896152599173,
 4.843020774701093,
 2.7577866756374108,
 4.90696942286709,
 2.8109819774353824,
 2.3959580326095877,
 3.781436578906325,
 3.840687694112261,
 3.6375475453413646,
 4.709000089445341,
 4.966572069809349,
 3.532561566166824,
 6.793550400014322,
 2.991509172297139,
 2.3235095823948813,
 5.687814035532691,
 2.6685691605486848,
 1.410872939087652,
 4.569879559073783,
 5.369083516638955,
 4.384096925146828,
 5.039730749426664,
 3.531809204814918,
 3.726442113285685,
 5.868838978848926,
 3.6841572399493385,
 1.8220027644542187,
 4.0117519967287025,
 3.1453353260252617,
 3.7435197690456827,
 5.310485966748522,
 3.103411544269188,
 3.885142672443723,
 5.948076917620928,
 3.0261372019319652,
 3.7954319946285184,
 4.25138570613127,
 5.044270257420849,
 3.6724026276537844,
 4.209287633829467,
 4.701589403366791,
 1.6

In [258]:
N

[267, 270, 269]

In [259]:
W = [1.1, 2.6, 1.3]

In [260]:
arm_to_play()

0

In [261]:
t = 4
stima_theta()

array([ 0.0395083 , -0.01284513,  0.00444665])

In [262]:
for i in range (0, num_arms):
    print(best_response(i))

TypeError: best_response() takes 0 positional arguments but 1 was given

In [ ]:
best_response(0)

In [ ]:
np.array(theta_stimato) - np.array(Th[best_response(0)].vector)

In [ ]:
np.array(A[0].arm_vector)

In [ ]:
p1 = np.matmul((np.array(theta_stimato) - np.array(Th[best_response(0)].vector)), np.array(A[0].arm_vector))
p1

In [ ]:
math.sqrt(2*beta(t, 1/t**3))

In [ ]:
np.sqrt(2*beta(t, 1/t**3))

In [ ]:
k = 1000
time = k
delta = 1/k**3


In [ ]:
n = np.abs(np.random.normal(0, 1))
n

In [ ]:
math.sqrt(math.log10(1/delta))

In [ ]:
d/2*math.log10(1 + (time*(l**2))/(n*d))

In [ ]:
math.sqrt(n/2)*m

In [ ]:
beta(k, 1/k**3)

In [ ]:
p2 = np.sqrt(2*beta(t, 1/t**3))*norm_2_mat(np.array(A[0].arm_vector), design_matrix(N) + np.random.normal(0, 1)*np.identity(d))

In [ ]:
calcola_U(0, 2)

In [ ]:
calcola_U(1, 2)

In [ ]:
calcola_U(2, 2)

In [ ]:
for i in range (0 , T//3 - t):
    a.pull_arm()
    b.pull_arm()
    c.pull_arm()
    t += 3

In [ ]:

stima_theta()

In [ ]:
for i in range (0, num_arms):
    print(best_response(i))

In [ ]:
for i in range (0, num_arms):
    print(np.matmul(A[i].arm_vector, Th[theta_vero[i]].vector))

In [ ]:
for i in range (0, num_arms):
    print(calcola_U(i, best_response(i))/100000000)